# Group 1

Fabio Ceruti- Fabio Martino - Alex Lucchini - Jacopo Biggiogera

This notebook contains code that allows to generated pictures with masked bounding boxes, bounding boxes filled with teh average pixel value of the context and to extract new pictures that are only the content of the bouding boxes

In [1]:
import os
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import shutil
from collections import defaultdict
from PIL import Image

In [1]:
def bb_mask(image, bboxes):
    '''This function identifies the bouding box and substitutes content with a 0'''
    if bboxes[0]==',':
        bboxes = bboxes[1:]
    bboxes = bboxes.split(',')
    start_x = int(bboxes[1])
    start_y = int(bboxes[0])
    end_x = start_x + int(bboxes[3])
    end_y = start_y + int(bboxes[2])
    if len(image.shape) == 2:
        image[start_x:end_x, start_y:end_y] = 0
    else:
        image[start_x:end_x, start_y:end_y, 0] = 0
        image[start_x:end_x, start_y:end_y, 1] = 0
        image[start_x:end_x, start_y:end_y, 2] = 0
    return image

In [3]:
def bb_removal(img_path, bboxes, savepath, filename):
    '''This function reads and image, removes the bounding box and saves it'''
    img = Image.open(img_path)
    img = np.array(img)
    for i in bboxes:
        img = bb_mask(img, i)
        
    #Saving image in specified directory with the same original name
    im = Image.fromarray(img)
    im.save(savepath+'/'+filename)
    return img

In [4]:
def bbox_cleaner(bbstr):
    '''This function reads the bouding box values form a dataframe'''
    bbstr = bbstr[1:-1]
    nb = bbstr.count('[')
    for i in range(nb):
        bbstr = bbstr.replace('[', '')
    bbstr = bbstr.split(']')
    return bbstr[:-1]

In [5]:
def bb_extract(image_path, bboxes, savepath, filename):
    '''This function extracts the content of a bounding box'''
    img = Image.open(image_path)
    img = np.array(img)
    start_x = int(bboxes[1])
    start_y = int(bboxes[0])
    end_x = start_x + int(bboxes[3])
    end_y = start_y + int(bboxes[2])
    if len(img.shape) == 2:
        box = img[start_x:end_x, start_y:end_y]
        box = (box - box.min())/(box.max()-box.min())
    else:
        box_1 = img[start_x:end_x, start_y:end_y, 0] 
        box_2 = img[start_x:end_x, start_y:end_y, 1] 
        box_3 = img[start_x:end_x, start_y:end_y, 2] 
        box_1 = (box_1 - box_1.min())/(box_1.max()-box_1.min())
        box_2 = (box_2 - box_2.min())/(box_2.max()-box_2.min())
        box_3 = (box_3 - box_3.min())/(box_3.max()-box_3.min())
        box = np.zeros((box_1.shape[0], box_1.shape[1], 3))
        box[:,:,0] = box_1
        box[:,:,1] = box_2
        box[:,:,2] = box_3
    im = Image.fromarray((box* 255).astype(np.uint8))
    im.save(savepath+'/'+filename)

# Removing bounding boxes

In [45]:
correct = pd.read_csv('correct_bboxes.csv')
correct.head()

,Unnamed: 0,pic_name,bbox,id,path_name,species
0,0,24a78d176d2408fae40725cdc48bd605.jpg,"[[281, 225, 118, 125]]",159413,/content/drive/MyDrive/Animals/Pandion haliaet...,Pandion haliaetus
1,1,8f87370536337294de8c925c3668ae2d.jpg,"[[234, 168, 337, 196]]",77529,/content/drive/MyDrive/Animals/Pachydiplax lon...,Pachydiplax longipennis
2,2,5d85146f3c7c25f71ca15ec9b4d8be54.jpg,"[[132, 110, 497, 520]]",275890,/content/drive/MyDrive/Animals/Junonia coenia/...,Junonia coenia
3,3,70958a293b1b2897248604c57fed16fb.jpg,"[[324, 310, 175, 160]]",2786,/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens
4,4,572b82540fca7c20c6246b0bd602c2f0.jpg,"[[247, 29, 295, 268]]",309373,/content/drive/MyDrive/Animals/Harmonia axyrid...,Harmonia axyridis


In [3]:
#Creating directories for each species
for spec in os.listdir('Animals'):
    if spec!= '.DS_Store':
        os.mkdir('Animals_masked/'+spec)
    

In [85]:
parent = 'Animals'
for bbox, path, name, species in zip(correct['bbox'], correct['path_name'], correct['pic_name'], correct['species']):
    p_vm = path[23:]
    new_location = 'Animals_masked/'+species+'/'
    bbox = bbox_cleaner(bbox)
    bb_removal(p_vm, bbox,new_location, name)

In [93]:
c = 0
for i in os.listdir('Animals_masked'):
    for t in os.listdir('Animals_masked/'+i):
        if t != '.DS_Store':
            c+=1
    print(f"{i}: {len(os.listdir('Animals_masked/'+i))}")
print(f'Total pictures: {c}')

Sciurus carolinensis: 969
Trachemys scripta elegans: 765
Junonia coenia: 890
Pandion haliaetus: 1794
Pachydiplax longipennis: 1576
Anolis carolinensis: 1093
Chelydra serpentina: 591
Spinus tristis: 1312
Procyon lotor: 889
Harmonia axyridis: 789
Picoides pubescens: 1503
Canis latrans: 1129
Total pictures: 13300


In [96]:
correct.groupby('species').count()['pic_name']

species
Anolis carolinensis          1092
Canis latrans                1128
Chelydra serpentina           591
Harmonia axyridis             789
Junonia coenia                890
Pachydiplax longipennis      1576
Pandion haliaetus            1794
Picoides pubescens           1503
Procyon lotor                 889
Sciurus carolinensis          969
Spinus tristis               1312
Trachemys scripta elegans     765
Name: pic_name, dtype: int64

# Filling bouding boxes with average color

In [ ]:
def avg_filling(img_path, savepath):
    image = Image.open(img_path)
    image = np.array(image)
    if len(image.shape) == 3:
        average_0 = image[:,:,0][image[:,:,0] != 0].mean()
        average_1 = image[:,:,1][image[:,:,1] != 0].mean()
        average_2 = image[:,:,2][image[:,:,2] != 0].mean()

        image[:,:,0][image[:,:,0] == 0] = average_0
        image[:,:,1][image[:,:,1] == 0] = average_1
        image[:,:,2][image[:,:,2] == 0] = average_2
    else:
        average_0 = image[:,:][image[:,:] != 0].mean()

        image[:,:][image[:,:] == 0] = average_0
        
    im = Image.fromarray(image)#(image* 255).astype(np.uint8))
    
    filename = img_path[15:]
    im.save(savepath+'/'+filename)

In [ ]:
#os.mkdir('Animals_masked_averaged')
for spec in os.listdir('Animals'):
    if spec!= '.DS_Store':
        os.mkdir('Animals_masked_averaged/'+spec)

In [ ]:
tv = pd.read_csv('train_val_data.csv')

In [ ]:
tv = pd.read_csv('train_val_data.csv')
for i in tqdm(tv['path']):
    part = i[7:]
    newpath ='Animals_masked'+part
    avg_filling(newpath, 'Animals_masked_averaged')

# Extracting only bounding boxes

In [6]:
bbcontent = pd.read_csv('no_context.csv')
bbcontent.head()

,Unnamed: 0,pic_name,id,bbox,path_name,species
0,0,2bf2dc73a77dbf99d1c92a64e49afec2.jpg,2199,"[251, 239, 91, 240]",/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens
1,1,be0ed422200ed45975ae3954bd5e7325.jpg,2200,"[249, 2, 227, 528]",/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens
2,2,dd09c0367b42e7890c127a33eb599043.jpg,2201,"[277, 124, 233, 475]",/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens
3,3,69cf3efe1d833d81b80e35b3456df7c5.jpg,2202,"[342, 193, 120, 288]",/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens
4,4,1e18c926ccb00243e6859b08ca67c089.jpg,2204,"[401, 309, 25, 58]",/content/drive/MyDrive/Animals/Picoides pubesc...,Picoides pubescens


In [101]:
#Creating directories for each species
for spec in os.listdir('Animals'):
    if spec!= '.DS_Store':
        os.mkdir('Animals_bbcontent/'+spec)

In [7]:
uniqueness = defaultdict(int)
extra_pics = []

parent = 'Animals'
for bbox, path, name, species in zip(bbcontent['bbox'], bbcontent['path_name'], bbcontent['pic_name'], bbcontent['species']):
    p_vm = path[23:]
    new_location = 'Animals_bbcontent/'+species+'/'
    bbox = bbox[1:-1].split(',')
    uniqueness[name]+=1
    new_name = uniqueness[name]
    if uniqueness[name]>1:
        name = name.split('.')
        name = name[0]+ str(new_name) +'.'+name[1]
        extra_pics.append(name)
    bb_extract(p_vm, bbox,new_location, name) 

In [126]:
c = 0
for i in os.listdir('Animals_bbcontent'):
    for t in os.listdir('Animals_bbcontent/'+i):
        if t != '.DS_Store':
            c+=1
    print(f"{i}: {len(os.listdir('Animals_bbcontent/'+i))}")
print(f'Total pictures: {c}')

Sciurus carolinensis: 1003
Trachemys scripta elegans: 1184
Junonia coenia: 927
Pandion haliaetus: 1999
Pachydiplax longipennis: 1607
Anolis carolinensis: 1112
Chelydra serpentina: 613
Spinus tristis: 1705
Procyon lotor: 1075
Harmonia axyridis: 899
Picoides pubescens: 1547
Canis latrans: 1270
Total pictures: 14941


In [150]:
#Saving data in a csv to call in the model
label_mapping = {'Anolis carolinensis': '0', 'Canis latrans':'1','Chelydra serpentina':'2', 'Harmonia axyridis':'3',
'Junonia coenia':'4', 'Pachydiplax longipennis':'5', 'Pandion haliaetus':'6', 'Picoides pubescens':'7',
'Procyon lotor':'8', 'Sciurus carolinensis':'8','Sciurus carolinensis':'9', 'Spinus tristis':'10', 'Trachemys scripta elegans':'11'}


In [151]:
path = []
species = []
label = []

parent = 'Animals_bbcontent/'
for spec in os.listdir('Animals_bbcontent'):
    for pic in os.listdir('Animals_bbcontent/'+spec):
        path.append(parent+spec+'/'+pic)
        species.append(spec)
        label.append(label_mapping[spec])
    

In [155]:
data_for_bbcontent_model = pd.DataFrame({'path':path, 'species':species, 'label':label})
data_for_bbcontent_model.to_csv('data_for_bbcontent_model.csv', index = False)